# Deep Research Agent Creation Notebook

This notebook demonstrates how to create a Deep Research Agent using the **Azure AI Agents Service** in **Azure AI Foundry**. It includes only the minimal steps required to provision an agent capable of performing grounded deep research with Bing and a deployed deep research model.

You will:

1. Load and validate required environment variables
2. Initialize the Azure AI Project client (Agents Service)
3. Resolve the Bing grounding connection
4. Initialize the Deep Research tool
5. Define concise agent instructions
6. Create the agent with tool definitions
7. Output the resulting `agent.id` for reuse (set it as `AGENT_ID` in your environment later)

> After creation, use the printed `AGENT_ID` when calling your API's `/research/submit` endpoint.


## 1. Required Environment Variables

The following environment variables must be defined (e.g., in a `.env` file or process environment):

| Variable | Description |
|----------|-------------|
| `PROJECT_ENDPOINT` | Azure AI Project endpoint URL |
| `BING_RESOURCE_NAME` | Name of the Bing grounding connection in the project |
| `DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME` | Deployment name of the deep research model (e.g., `o3-deep-research`) |
| `MODEL_DEPLOYMENT_NAME` | Deployment name of the base model the agent will use (e.g., `gpt-4o`) |

If you haven’t set them, create a `.env` file in the repo root before running the next cell.

In [8]:
# 2. Load environment variables and validate
import os
from dotenv import load_dotenv

required_vars = [
    "PROJECT_ENDPOINT",
    "BING_RESOURCE_NAME",
    "DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME",
    "MODEL_DEPLOYMENT_NAME"
]

load_dotenv('../api/.env', override=True)
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    raise ValueError(f"Missing required environment variables: {missing}")
else:
    print("All required environment variables are set.")

All required environment variables are set.


## 3. Initialize Azure AI Project Client
This client lets us interact with the Agents service (threads, messages, agents, tools, connections). We authenticate with `DefaultAzureCredential`, so be sure your Azure CLI or managed identity context is valid.

In [9]:
# 3. Initialize Azure AI Project Client
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)
print("Project client initialized.")

Project client initialized.


## 4. Resolve Bing Grounding Connection
We fetch the connection by name (set in `BING_RESOURCE_NAME`). This provides the `id` needed when constructing the Deep Research tool.

In [10]:
# 4. Resolve Bing Grounding Connection ID
conn_id = project_client.connections.get(name=os.environ["BING_RESOURCE_NAME"]).id
print("Bing connection id:", conn_id)

Bing connection id: /subscriptions/3a72b9f9-cb8d-44de-827e-ade93ff2dab5/resourceGroups/rg-deepresearch-testing/providers/Microsoft.CognitiveServices/accounts/deep-research-testing-foundry/projects/deep-research-testing-project/connections/rutzscoagents


## 5. Initialize Deep Research Tool
We construct `DeepResearchTool` providing grounding connection id and the deep research model deployment. This produces tool definitions consumed by the agent creation call.

In [11]:
# 5. Initialize Deep Research Tool
from azure.ai.agents.models import DeepResearchTool

deep_research_tool = DeepResearchTool(
    bing_grounding_connection_id=conn_id,
    deep_research_model=os.environ["DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME"],
)
print("Deep Research tool initialized.")

Deep Research tool initialized.


## 6. Define Agent Instructions
These instructions guide the Deep Research agent to produce structured, evidence-based reports with citations and analytical depth.

In [12]:
# 6. Define Agent Instructions
instructions = """You are tasked with conducting a detailed, data-driven analysis based on the user's query across any research domain. Your role is to deliver structured, evidence-based insights tailored to inform decision-making or deepen understanding within the specified topic in the form of a well-organized report.  
  
## Key Objectives  
1. **Data-Driven Insights**: Incorporate specific and measurable information, such as trends, statistics, comparisons, and actionable outcomes relevant to the topic (e.g., industry growth rates, performance metrics, user adoption rates, societal impact).       
2. **Reliable Sources**: Prioritize credible, up-to-date references, such as peer-reviewed studies, official reports, industry analyses, or government publications.    
3. **Citations and Metadata**: Include inline citations for all claims and provide full metadata for the sources used, ensuring traceability and reliability.  
  
## Approach  
- Be analytical, avoiding generalities, and focus on creating a well-structured argument supported by data.    
- Each section should leverage evidence to support conclusions and provide insights that could be used for strategic planning, policy development, or further exploration of the topic.
"""

## 7. Create the Deep Research Agent
We now call `create_agent` providing the model deployment name, instructions, and tool definitions.

In [13]:
# 7. Create the Deep Research Agent
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="deep-research-agent",
    instructions=instructions,
    tools=deep_research_tool.definitions,
)
print("Agent created.")

Agent created.


## 8. Output Agent ID
Store or export this value (e.g., set `AGENT_ID` in your environment) for subsequent research submissions.

In [14]:
# 8. Print Agent ID
print("AGENT_ID=", agent.id)
agent.id

AGENT_ID= asst_aMft24CsmWMChmkb30ai30tD


'asst_aMft24CsmWMChmkb30ai30tD'